In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans

file = r'E:\machin-prj\my_dataframe5.xlsx'
# 1. 데이터 로드 (my_dataframe5.xlsx)
df = pd.read_excel(file)


In [2]:
# # 1) 출발지 및 필터링
# start_station = '경복궁역'
# df_food = df[
#     (df['closest_subway_station'] == start_station) &
#     (df['cat1'] == '음식')
# ].copy()

# # 2) KMeans 클러스터링
# coords = df_food[['mapx','mapy']].values
# kmeans = KMeans(n_clusters=5, random_state=42).fit(coords)
# df_food['cluster'] = kmeans.labels_

# # 3) 각 군집의 대표 음식점 인덱스 수집
# rep_indices = []
# for i in range(5):
#     pts = df_food[df_food['cluster'] == i]
#     centroid = kmeans.cluster_centers_[i]
#     dists = np.linalg.norm(pts[['mapx','mapy']].values - centroid, axis=1)
#     rep_idx = pts.index[np.argmin(dists)]
#     rep_indices.append(rep_idx)

# # 4) 교통수단 라인 추출 함수
# def get_lines(row):
#     vals = row[['line1_x','line2_x','line3_x','line4_x','정류장번호']].dropna().astype(str)
#     return set(vals.values)

# # 5) 출발지 정보 로드
# start_row = df[df['closest_subway_station'] == start_station].iloc[0]
# current_coord = np.array([start_row.mapx, start_row.mapy])
# current_lines = get_lines(start_row)

# # 6) 그리디 순회 경로 생성
# remaining = rep_indices.copy()
# route = []
# while remaining:
#     candidates = [i for i in remaining if current_lines & get_lines(df_food.loc[i])]
#     if not candidates:
#         candidates = remaining.copy()
#     dists = [
#         np.linalg.norm(np.array([df_food.loc[i].mapx, df_food.loc[i].mapy]) - current_coord)
#         for i in candidates
#     ]
#     next_idx = candidates[int(np.argmin(dists))]
#     row = df_food.loc[next_idx]
#     shared = current_lines & get_lines(row)
#     transport = next(iter(shared)) if shared else next(iter(get_lines(row)))
#     route.append({'restaurant': row.title, 'transport': transport})
#     current_coord = np.array([row.mapx, row.mapy])
#     current_lines = get_lines(row)
#     remaining.remove(next_idx)

# # 7) 결과를 열로 저장
# cols = {}
# for i, step in enumerate(route, 1):
#     cols[f'음식점{i}']   = step['restaurant']
#     cols[f'이동수단{i}'] = step['transport']

# route_df = pd.DataFrame([cols])

In [3]:
# route_df

In [4]:
# df

In [5]:
import pandas as pd
from math import radians, sin, cos, sqrt, asin

# Haversine 함수 정의
def haversine(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat/2)**2 + cos(lat1)*cos(lat2)*sin(dlon/2)**2
    return 2 * 6371 * asin(sqrt(a))


# 1) 음식 카테고리 필터링
df_food = df[df['cat1'] == '음식'].copy()

# 2) 경복궁역 좌표
station_lat, station_lon = 37.579617, 126.977041

# 3) 경복궁역에서 음식점까지 거리 계산
df_food['distance'] = df_food.apply(
    lambda r: haversine(station_lat, station_lon, r['mapy'], r['mapx']), axis=1
)

# 4) 가장 가까운 10개 음식점 추출
top10 = df_food.nsmallest(10, 'distance').reset_index(drop=True)
top10

,addr1,cat1,cat2,cat3,mapx,mapy,nearest_station,closest_subway_station,matched_station_name,정류장번호,line1_x,line2_x,line3_x,line4_x,title,distance
0,서울특별시 종로구 삼청로 30,음식,음식점,한식,126.979904,37.578595,경복궁,안국역,경복궁,GGB100000022,03호선,NaN,NaN,NaN,두레 국립현대미술관서울점,0.276722
1,서울특별시 종로구 삼청로 30,음식,음식점,한식,126.979904,37.578595,경복궁,안국역,경복궁,GGB100000022,03호선,NaN,NaN,NaN,국립현대미술관 서울,0.276722
2,서울특별시 종로구 삼청로 24,음식,음식점,카페/전통찻집,126.979848,37.578103,경복궁,안국역,경복궁,GGB100000022,03호선,NaN,NaN,NaN,널담은공간,0.299242
3,서울특별시 종로구 삼청로 22-7,음식,음식점,한식,126.980048,37.578031,경복궁,안국역,경복궁,GGB100000022,03호선,NaN,NaN,NaN,스미스가 좋아하는 한옥,0.318347
4,서울특별시 종로구 북촌로5길 78,음식,음식점,한식,126.980637,37.580092,덕성여중고,안국역,덕성여중고,GGB100000035,03호선,NaN,NaN,NaN,황생가칼국수,0.321261
5,서울특별시 종로구 삼청로 54,음식,음식점,서양식,126.980698,37.580744,덕성여중고,안국역,덕성여중고,GGB100000035,03호선,NaN,NaN,NaN,더레스토랑,0.345799
6,서울특별시 종로구 자하문로10길 28,음식,음식점,카페/전통찻집,126.973049,37.579350,경복궁역1번출구,경복궁역,경복궁역1번출구,GGB100000019,03호선,NaN,NaN,NaN,라 카페 갤러리,0.353047
7,서울특별시 종로구 삼청로 14,음식,음식점,서양식,126.979804,37.577300,경복궁,안국역,경복궁,GGB100000022,03호선,NaN,NaN,NaN,두가헌,0.354465
8,서울특별시 종로구 삼청로 14,음식,음식점,서양식,126.979804,37.577300,경복궁,안국역,경복궁,GGB100000022,03호선,NaN,NaN,NaN,갤러리 현대,0.354465
9,서울특별시 종로구 삼청로2길 40,음식,음식점,카페/전통찻집,126.981134,37.580119,덕성여중고,안국역,덕성여중고,GGB100000035,03호선,NaN,NaN,NaN,거울한옥 미러룸,0.365011


In [6]:
stage_names = top10["title"].tolist()  # ()를 붙여야 실제 리스트가 반환됩니다.
print(stage_names)

['두레 국립현대미술관서울점', '국립현대미술관 서울', '널담은공간', '스미스가 좋아하는 한옥', '황생가칼국수', '더레스토랑', '라 카페 갤러리', '두가헌', '갤러리 현대', '거울한옥 미러룸']


In [7]:
# 1) '음식' 카테고리 필터링
df_food = df[df['cat1'] == '음식'].copy()

# 2) 경복궁역 좌표
station_lat, station_lon = 37.579617, 126.977041

# 3) 경복궁역으로부터 거리 계산
df_food['distance'] = df_food.apply(
    lambda r: haversine(station_lat, station_lon, r['mapy'], r['mapx']), axis=1
)

# 4) 가장 가까운 10개 음식점 추출
top10 = df_food.nsmallest(10, 'distance').reset_index(drop=True)

# 5) stage_names 설정
stage_names = top10["title"].tolist()

# 추천 대상에서 top10 제외
df_candidates = df_food.drop(top10.index)

# 6) 각 스테이지별 2단계 추천 경로 생성 (5~10km 반경)
routes = []
for idx, row in top10.iterrows():
    stage = stage_names[idx]
    # 이동수단1, 출발/도착 설정
    if idx % 2 == 0:
        mode1, start1, end1 = '지하철', '경복궁역', row['closest_subway_station']
    else:
        mode1, start1, end1 = '버스', row['정류장번호'], row['matched_station_name']
    
    # 1차 추천: 5~10km 내 5개
    df_candidates['dist1'] = df_candidates.apply(
        lambda r: haversine(row['mapy'], row['mapx'], r['mapy'], r['mapx']), axis=1
    )
    rec1_df = df_candidates[(df_candidates['dist1'] >= 5) & (df_candidates['dist1'] <= 10)]
    rec1_df = rec1_df.nsmallest(5, 'dist1')
    rec1_list = rec1_df['title'].tolist()
    
    # 2차 추천: rec1_df 첫 번째 음식점 기준 5~10km 내 5개
    if not rec1_df.empty:
        rec1_first = rec1_df.iloc[0]
        df_candidates2 = df_candidates.drop(rec1_df.index)
        df_candidates2['dist2'] = df_candidates2.apply(
            lambda r: haversine(rec1_first['mapy'], rec1_first['mapx'], r['mapy'], r['mapx']), axis=1
        )
        rec2_df = df_candidates2[(df_candidates2['dist2'] >= 5) & (df_candidates2['dist2'] <= 10)]
        rec2_df = rec2_df.nsmallest(5, 'dist2')
        rec2_list = rec2_df['title'].tolist()
        # 이동수단2 설정 (mode1과 반대)
        mode2 = '버스' if mode1 == '지하철' else '지하철'
        if mode2 == '지하철':
            start2 = rec1_first['closest_subway_station']
            end2 = rec2_df.iloc[0]['closest_subway_station'] if not rec2_df.empty else None
        else:
            start2 = rec1_first['정류장번호']
            end2 = rec2_df.iloc[0]['matched_station_name'] if not rec2_df.empty else None
    else:
        mode2, start2, end2, rec2_list = None, None, None, []
    
    routes.append({
        '단계': stage,
        '이동수단1': mode1,
        '타는곳1': start1,
        '내리는곳1': end1,
        '추천음식점1': ', '.join(rec1_list),
        '이동수단2': mode2,
        '타는곳2': start2,
        '내리는곳2': end2,
        '추천음식점2': ', '.join(rec2_list)
    })

# 결과 DataFrame 표시
result_df = pd.DataFrame(routes)
result_df

,단계,이동수단1,타는곳1,내리는곳1,추천음식점1,이동수단2,타는곳2,내리는곳2,추천음식점2
0,두레 국립현대미술관서울점,지하철,경복궁역,안국역,"춘향미엔, 공기(Gongi), 공기(Gongi), 공기(Gongi), 철길왕갈비살",버스,GGB103000011,용산구청,"유진막국수, 유진막국수, 두레 국립현대미술관서울점, 국립현대미술관 서울, 황생가칼국수"
1,국립현대미술관 서울,버스,GGB100000022,경복궁,"춘향미엔, 공기(Gongi), 공기(Gongi), 공기(Gongi), 철길왕갈비살",지하철,행당역,이태원역,"유진막국수, 유진막국수, 두레 국립현대미술관서울점, 국립현대미술관 서울, 황생가칼국수"
2,널담은공간,지하철,경복궁역,안국역,"마포원조떡볶이, 마포원조떡볶이, 알페도, 알페도, 제스티살룬 연남",버스,ICB113000001,안국동,"인사동 촌, 전통다원, 경인미술관, 널담은공간, 스미스가 좋아하는 한옥"
3,스미스가 좋아하는 한옥,버스,GGB100000022,경복궁,"마포원조떡볶이, 마포원조떡볶이, 제스티살룬 연남, 벤스쿠키 이태원점, 벤스쿠키 이태원점",지하철,마포역,안국역,"인사동 촌, 전통다원, 경인미술관, 널담은공간, 스미스가 좋아하는 한옥"
4,황생가칼국수,지하철,경복궁역,안국역,"이모네순대국, 춘향미엔, 모래내곱창, 파스타얼론, 파스타얼론",버스,GGB277102791,세종문화회관,"길복정, 길복정, 황생가칼국수, 졍동국시, 평안도만두집"
5,더레스토랑,버스,GGB100000035,덕성여중고,"락희안 가좌본점, 카페미우, 연남장, 몽탄, 몽탄",지하철,가좌역,종각역,"광화문등심, 중앙갈치식당, 중앙갈치식당, 그랜드컨벤션센터, 그랜드컨벤션센터"
6,라 카페 갤러리,지하철,경복궁역,경복궁역,"셰프테이너, GOO STK 733, 셰프테이너, GOO STK 733, 이화감자국",버스,GGB102000058,혜화역2번출구.마로니에공원,"조스바스켓, 도성, 도성, 도성, 도성"
7,두가헌,버스,GGB100000022,경복궁,"청춘극장, 청춘극장, 소이연남, 메종브리크, 땅코참숯구이 본점",지하철,행당역,서울역,"태향, 장군갈비, 루이, 성북동돼지갈비, 두가헌"
8,갤러리 현대,지하철,경복궁역,안국역,"청춘극장, 청춘극장, 소이연남, 메종브리크, 땅코참숯구이 본점",버스,GGB103000034,남대문경찰서,"태향, 장군갈비, 루이, 성북동돼지갈비, 두가헌"
9,거울한옥 미러룸,버스,GGB100000035,덕성여중고,"마포양지설렁탕, 엄마손떡볶이, 골목냉면, 연남장, 락희안 가좌본점",지하철,공덕역,종로3가역,"돌고래횟집, 돌고래횟집, 궁전백반삼겹살, 함흥냉면, 유스커피"


In [8]:
import pandas as pd
import numpy as np
from sklearn.neighbors import BallTree
from math import radians

In [9]:
df_food = df[df['cat1'] == '음식'].reset_index(drop=True)

# BallTree 생성 (haversine 거리)
coords_rad = np.radians(df_food[['mapy', 'mapx']].values)
tree = BallTree(coords_rad, metric='haversine')

# 추천 함수: low_km <= distance <= high_km, 최대 k개
def recommend_km_range(lat, lon, low_km, high_km, k):
    point = np.radians([[lat, lon]])
    idxs = tree.query_radius(point, r=high_km/6371)[0]
    if len(idxs) == 0:
        return pd.DataFrame(columns=df_food.columns)
    dists = tree.query(point, k=len(idxs))[0][0] * 6371
    valid_ids = idxs[(dists >= low_km)]
    valid_ids = valid_ids[:k]
    return df_food.iloc[valid_ids]

# 초기 스테이지: 경복궁역 기준 0~10km에서 10개
station_lat, station_lon = 37.579617, 126.977041
initial = recommend_km_range(station_lat, station_lon, 0, 10, 10)

# 2단계 ML 경로 추천 (boarding != alighting 보장)
routes = []
for idx, row in initial.iterrows():
    stage = row['title']
    # 이동수단1 결정
    if idx % 2 == 0:
        mode1 = '지하철'
        start1 = '경복궁역'
        end1 = row['closest_subway_station']
    else:
        mode1 = '버스'
        start1 = row['정류장번호']
        end1 = row['matched_station_name']
    # 1차 추천 (5~10km, up to 5개)
    rec1_df = recommend_km_range(row['mapy'], row['mapx'], 5, 10, 5)
    rec1_list = rec1_df['title'].tolist()
    # 이동수단2 (반대)
    mode2 = '버스' if mode1 == '지하철' else '지하철'
    # 2차 추천: 첫 추천 지점 기준
    if not rec1_df.empty:
        first = rec1_df.iloc[0]
        # find rec2 candidates
        rec2_df = recommend_km_range(first['mapy'], first['mapx'], 5, 10, 10)
        # filter out same boarding and alighting
        if mode2 == '지하철':
            rec2_df = rec2_df[rec2_df['closest_subway_station'] != first['closest_subway_station']]
            start2 = first['closest_subway_station']
            end2 = rec2_df.iloc[0]['closest_subway_station'] if not rec2_df.empty else None
        else:
            rec2_df = rec2_df[rec2_df['matched_station_name'] != first['matched_station_name']]
            start2 = first['matched_station_name']
            end2 = rec2_df.iloc[0]['matched_station_name'] if not rec2_df.empty else None
        rec2_list = rec2_df['title'].tolist()[:5]
    else:
        start2 = end2 = mode2 = None
        rec2_list = []
    routes.append({
        '단계': stage,
        '이동수단1': mode1,
        '타는곳1': start1,
        '내리는곳1': end1,
        '추천음식점1': ', '.join(rec1_list),
        '이동수단2': mode2,
        '타는곳2': start2,
        '내리는곳2': end2,
        '추천음식점2': ', '.join(rec2_list)
    })

# 결과 출력
result_df = pd.DataFrame(routes)

In [10]:
result_df

,단계,이동수단1,타는곳1,내리는곳1,추천음식점1,이동수단2,타는곳2,내리는곳2,추천음식점2
0,유림,버스,GGB115000151,염창동새마을금고,"서글렁탕, 멘슈, 마도, 희래등, 희래등",지하철,샛강역,발산역,"등촌샤브칼국수 등촌, 이가바지락칼국수, 이가바지락칼국수, 이조면옥, 모센즈 스위트"
1,유림,지하철,경복궁역,염창역,"서글렁탕, 멘슈, 마도, 희래등, 희래등",버스,진주아파트,서울호서전문학교.등촌주공3단지아파트,"등촌샤브칼국수 등촌, 이가바지락칼국수, 이가바지락칼국수, 이조면옥, 모센즈 스위트"
2,하오아,지하철,경복궁역,염창역,"서글렁탕, 멘슈, 진남회집, 청해진, 용궁횟집",버스,진주아파트,서울호서전문학교.등촌주공3단지아파트,"등촌샤브칼국수 등촌, 이가바지락칼국수, 이가바지락칼국수, 이조면옥, 모센즈 스위트"
3,이한스시,버스,GGB118000080,양남동로타리,"중앙식당, 미자식당, 충남식당, 노량진수산물도매시장, 진남회집",지하철,노량진역,회현역,"후암동108계단, 마포유가궁중족발, 마포소문난원조족발, 더이터리(포 포인츠 바이 쉐..."
4,곱,버스,GGB118000204,문래우체국.문래예술공단,"진남회집, 청해진, 또순이, 또순이, 청해진",지하철,노량진역,회현역,"후암동108계단, 마포유가궁중족발, 마포소문난원조족발, 더이터리(포 포인츠 바이 쉐..."
5,금수강산,버스,GGB118000077,당산동진로아파트,"멘슈, 서글렁탕, 멘슈, 서글렁탕, 멘슈",지하철,샛강역,가양역,"이가바지락칼국수, 이조면옥, 모센즈 스위트, 초원오리농장, 초원오리농장"
6,가정식당,지하철,경복궁역,문래역,"충남식당, 초록바구니, 사담, 동작구름카페, 장충족발",버스,여의상류IC(경유),후암동종점,"후암동108계단, 마포유가궁중족발, 마포소문난원조족발, 더이터리(포 포인츠 바이 쉐..."
7,바다회장어구이,지하철,경복궁역,문래역,"진남회집, 청해진, 또순이, 또순이, 청해진",버스,여의상류IC(경유),후암동종점,"후암동108계단, 마포유가궁중족발, 마포소문난원조족발, 더이터리(포 포인츠 바이 쉐..."
8,금수강산,지하철,경복궁역,영등포구청역,"멘슈, 서글렁탕, 멘슈, 서글렁탕, 멘슈",버스,진주아파트,SBS공개홀,"이가바지락칼국수, 이조면옥, 모센즈 스위트, 초원오리농장, 초원오리농장"
9,가정식당,버스,GGB118000078,당산동진로아파트,"충남식당, 초록바구니, 사담, 동작구름카페, 장충족발",지하철,노량진역,회현역,"후암동108계단, 마포유가궁중족발, 마포소문난원조족발, 더이터리(포 포인츠 바이 쉐..."


In [11]:
file_review = r'E:\machin-prj\review_data_인천_행정동추가.csv'
df_review = pd.read_csv(file_review, encoding='utf-8')
df_review.head(2)

,검색어,장소명,평점,주소,리뷰닉네임,별점,날짜,리뷰내용,title,addr1,contentid,contenttypeid,유형,연도월,new_cat,행정동명,자치구,경도,위도
0,감중공원,감중공원,3.3,인천광역시 서구 가정로98번길 18,헷,5.0,2024.11.14.,NaN,감중공원,인천광역시 서구 가정로98번길 18,2767573.0,12.0,관광지,2024-11,힐링,가좌동,서구,126.662197,37.491604
1,갑곶돈대,갑곶돈대,4.4,인천광역시 강화군 강화읍 갑곳리 1022,병권💚,5.0,2025.04.06.,시설이 좋아요 😀 ㅎㅎ,갑곶돈대,인천광역시 강화군 강화읍 해안동로1366번길 18,1254680.0,12.0,관광지,2025-04,문화,강화읍,강화군,126.488237,37.751891


sent_text: 텍스트만으로 분석한 감정

sent_score: 별점만으로 분류한 감정 (45점 긍정, 12점 부정, 3점 중립)

sentiment: 텍스트가 없을 때 sent_score, 텍스트가 있을 때 sent_text를 사용

In [12]:
# 2. 텍스트 및 별점 컬럼 지정
text_col = '리뷰내용'
score_col = '별점'

# 3. 사전 기반 감정어 사전
pos_words = ["좋다", "최고", "만족", "추천", "행복", "감사", "친절", "편리", "맛있"]
neg_words = ["나쁘다", "불만", "최악", "짜증", "불편", "실망", "불친절", "별로"]

# 4. 텍스트 기반 감정 분석 함수
def get_text_sentiment(text):
    text = str(text)
    pos = sum(text.count(w) for w in pos_words)
    neg = sum(text.count(w) for w in neg_words)
    if pos > neg:
        return "positive"
    elif neg > pos:
        return "negative"
    else:
        return "neutral"

# 5. 별점 기반 감정 분류 함수
def get_score_sentiment(score):
    try:
        s = float(score)
    except:
        return "neutral"
    if s >= 4:
        return "positive"
    elif s <= 2:
        return "negative"
    else:
        return "neutral"

# 6. 감정 컬럼 생성
df_review['sent_text'] = df_review[text_col].apply(get_text_sentiment)
df_review['sent_score'] = df_review[score_col].apply(get_score_sentiment)

# 7. 최종 감정: 텍스트가 없거나 NaN인 경우 별점 기반 감정 사용
def combine_sentiment(row):
    txt = row[text_col]
    if pd.isna(txt) or str(txt).strip() == "":
        return row['sent_score']
    else:
        return row['sent_text']

df_review['sentiment'] = df_review.apply(combine_sentiment, axis=1)

# 8. 분포 요약 및 샘플
sent_counts = df_review['sentiment'].value_counts().reset_index()
sent_counts.columns = ['sentiment', 'count']

df_review.head(2)


,검색어,장소명,평점,주소,리뷰닉네임,별점,날짜,리뷰내용,title,addr1,...,유형,연도월,new_cat,행정동명,자치구,경도,위도,sent_text,sent_score,sentiment
0,감중공원,감중공원,3.3,인천광역시 서구 가정로98번길 18,헷,5.0,2024.11.14.,NaN,감중공원,인천광역시 서구 가정로98번길 18,...,관광지,2024-11,힐링,가좌동,서구,126.662197,37.491604,neutral,positive,positive
1,갑곶돈대,갑곶돈대,4.4,인천광역시 강화군 강화읍 갑곳리 1022,병권💚,5.0,2025.04.06.,시설이 좋아요 😀 ㅎㅎ,갑곶돈대,인천광역시 강화군 강화읍 해안동로1366번길 18,...,관광지,2025-04,문화,강화읍,강화군,126.488237,37.751891,neutral,positive,neutral


In [13]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans

file = r'E:\machin-prj\my_dataframe5.xlsx'
# 1. 데이터 로드 (my_dataframe5.xlsx)
df = pd.read_excel(file)


In [14]:

df = pd.merge(df, df_review[['addr1', 'sent_score']], how='left', on='addr1')
df.head(2)

,addr1,cat1,cat2,cat3,mapx,mapy,nearest_station,closest_subway_station,matched_station_name,정류장번호,line1_x,line2_x,line3_x,line4_x,title,sent_score
0,서울특별시 송파구 새말로 114 (문정동),음식,음식점,한식,127.126643,37.482681,문정법조단지.건영아파트(중),장지역,문정법조단지.건영아파트(중),GGB123000013,08호선,NaN,NaN,NaN,푼주,NaN
1,서울특별시 송파구 새말로 114 (문정동),음식,음식점,한식,127.126643,37.482681,문정법조단지.건영아파트(중),장지역,문정법조단지.건영아파트(중),GGB123000014,08호선,NaN,NaN,NaN,푼주,NaN


In [15]:
df = df[df['sent_score'] != 'negative'].reset_index(drop=True)
df.head(2)

,addr1,cat1,cat2,cat3,mapx,mapy,nearest_station,closest_subway_station,matched_station_name,정류장번호,line1_x,line2_x,line3_x,line4_x,title,sent_score
0,서울특별시 송파구 새말로 114 (문정동),음식,음식점,한식,127.126643,37.482681,문정법조단지.건영아파트(중),장지역,문정법조단지.건영아파트(중),GGB123000013,08호선,NaN,NaN,NaN,푼주,NaN
1,서울특별시 송파구 새말로 114 (문정동),음식,음식점,한식,127.126643,37.482681,문정법조단지.건영아파트(중),장지역,문정법조단지.건영아파트(중),GGB123000014,08호선,NaN,NaN,NaN,푼주,NaN


In [16]:
import pandas as pd
from math import radians, sin, cos, sqrt, asin

# Haversine 함수 정의
def haversine(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat/2)**2 + cos(lat1)*cos(lat2)*sin(dlon/2)**2
    return 2 * 6371 * asin(sqrt(a))


# 1) 음식 카테고리 필터링
df_food = df[df['cat1'] == '음식'].copy()

# 2) 인천천역 좌표
station_lat, station_lon = 37.476919, 126.616981

# 3) 인천역에서 음식점까지 거리 계산
df_food['distance'] = df_food.apply(
    lambda r: haversine(station_lat, station_lon, r['mapy'], r['mapx']), axis=1
)


# 긍정 리뷰 장소에서 가까운 10개 추출
top10_positive = df_food[df_food['sent_score'] == 'positive'].nsmallest(10, 'distance').reset_index(drop=True)

# 중립 리뷰 장소에서 가까운 10개 추출
top10_neutral = df_food[df_food['sent_score'] == 'neutral'].nsmallest(10, 'distance').reset_index(drop=True)

# # 4) 가장 가까운 10개 음식점 추출
# top10 = df_food.nsmallest(10, 'distance').reset_index(drop=True)
# top10

In [17]:
top10_positive.head(2)

,addr1,cat1,cat2,cat3,mapx,mapy,nearest_station,closest_subway_station,matched_station_name,정류장번호,line1_x,line2_x,line3_x,line4_x,title,sent_score,distance
0,인천광역시 중구 차이나타운로44번길 31-3,음식,음식점,한식,126.617762,37.475556,인천역(차이나타운),인천역,동인천역북광장,GGB162000136,01호선,수인선,NaN,NaN,신승반점,positive,0.166531
1,인천광역시 중구 차이나타운로44번길 31-3,음식,음식점,한식,126.617762,37.475556,인천역(차이나타운),인천역,동인천역북광장,GGB162000136,01호선,수인선,NaN,NaN,신승반점,positive,0.166531


In [ ]:
import pandas as pd
import numpy as np
from sklearn.neighbors import BallTree

# 1) '음식' 카테고리 + sentiment가 'positive'인 데이터만 필터링
df_food = df[(df['cat1'] == '음식') & (df['sent_score'] == 'positive')].reset_index(drop=True)



,단계,이동수단1,타는곳1,내리는곳1,추천음식점1,이동수단2,타는곳2,내리는곳2,추천음식점2
0,송도 한옥마을,지하철,센트럴파크역,센트럴파크역,"포크, 포크, 포크, 애월몽, 포크",버스,송도테크노파크IT센터,인천교회,"송돗골, 송돗골, 송돗골, 송돗골, 송돗골"
1,송도 한옥마을,지하철,센트럴파크역,센트럴파크역,"포크, 포크, 포크, 애월몽, 포크",버스,송도테크노파크IT센터,인천교회,"송돗골, 송돗골, 송돗골, 송돗골, 송돗골"
2,송도 한옥마을,버스,GGB164000397,센트럴파크역,"포크, 포크, 포크, 애월몽, 포크",지하철,테크노파크역,송도역,"송돗골, 송돗골, 송돗골, 송돗골, 송돗골"
3,송도 한옥마을,버스,GGB164000397,센트럴파크역,"포크, 포크, 포크, 애월몽, 포크",지하철,테크노파크역,송도역,"송돗골, 송돗골, 송돗골, 송돗골, 송돗골"
4,송도 한옥마을,지하철,센트럴파크역,센트럴파크역,"포크, 포크, 포크, 애월몽, 포크",버스,송도테크노파크IT센터,인천교회,"송돗골, 송돗골, 송돗골, 송돗골, 송돗골"
5,송도 한옥마을,버스,GGB164000397,센트럴파크역,"포크, 포크, 포크, 애월몽, 포크",지하철,테크노파크역,송도역,"송돗골, 송돗골, 송돗골, 송돗골, 송돗골"
6,송도 한옥마을,지하철,센트럴파크역,센트럴파크역,"포크, 포크, 포크, 애월몽, 포크",버스,송도테크노파크IT센터,인천교회,"송돗골, 송돗골, 송돗골, 송돗골, 송돗골"
7,송도 한옥마을,버스,GGB164000397,센트럴파크역,"포크, 포크, 포크, 애월몽, 포크",지하철,테크노파크역,송도역,"송돗골, 송돗골, 송돗골, 송돗골, 송돗골"
8,송도 한옥마을,지하철,센트럴파크역,센트럴파크역,"포크, 포크, 포크, 애월몽, 포크",버스,송도테크노파크IT센터,인천교회,"송돗골, 송돗골, 송돗골, 송돗골, 송돗골"
9,송도 한옥마을,버스,GGB164000397,센트럴파크역,"포크, 포크, 포크, 애월몽, 포크",지하철,테크노파크역,송도역,"송돗골, 송돗골, 송돗골, 송돗골, 송돗골"
